In [1]:
from lib.db import read_mongodb_to_dataframe, rewrite_data
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import plotly.express as px
import lib.eda as eda
from datetime import date

In [2]:
properties_db = read_mongodb_to_dataframe('real_estate_db', 'properties', no_id=False)
#properties_details_db = read_mongodb_to_dataframe('real_estate_db', 'properties_details', no_id=False)

In [3]:
# do backup
properties_db.to_csv(f'./backup/properties_db_{date.today().strftime("%d%m%Y")}.csv')

In [4]:
properties = properties_db.copy()
properties['_id']=properties['_id'].apply(pd.to_numeric, errors='coerce')

In [5]:
print(len(properties))
duplicate_properties = properties.groupby('_id').filter(lambda x: len(x) > 1).sort_values('_id')
print(len(duplicate_properties))
#temp = temp[temp['_id'] == 9005330]

51159
0


In [6]:
duplicate_properties.head(20)

,_id,,add_date,address,city,company,description,img_src,link,plex-revenue,...,vendue,revenu,surface,longitude,latitude,construction_year,municipal_taxes,school_taxes,unites,geolocation


In [7]:
duplicate_properties.to_csv('duplicate_properties.csv')

In [8]:
import math

elt=None
duplicate_properties_ok = []
for i,row in duplicate_properties.iterrows():
    change = False
    if elt is not None:
        if int(row['_id']) == int(elt['_id']):
            is_old = len(row['update_at']) > len(elt['update_at'])
            for key, value in row.items():
                if elt[key] != value:
                    if (isinstance(elt[key], str)\
                        or(key in ('surface','revenu','plex-revenue') and elt[key] is float and (math.isnan(elt[key])\
                            and value is float and not math.isnan(value)))\
                        or (key not in ('surface','revenu','plex-revenue')))\
                        and is_old:
                        change = True 
                        elt[key] = value
            change = True if len(duplicate_properties_ok) == 0 or elt not in duplicate_properties_ok else change
            
    if change:
        elt.pop('')
        elt['_id'] = int(elt['_id'])
        duplicate_properties_ok.append(elt)
    elt = row.to_dict()

duplicate_properties_ok = pd.DataFrame(duplicate_properties_ok)
                

In [9]:
#temp_fix = temp_ar.groupby('_id').filter(lambda x: len(x) > 1).sort_values('_id')
#print(len(temp_fix))

In [10]:
if len(duplicate_properties_ok) > 0:
    duplicate_properties_ok_ids = duplicate_properties_ok['_id']

In [11]:
properties_with_id_str = properties_db[properties_db.apply(lambda x: isinstance(x['_id'], str), axis=1)].copy()

In [12]:
len(properties_with_id_str)

0

In [13]:
if len(duplicate_properties_ok) > 0 and len(properties_with_id_str) > 0:
    properties_with_id_str['_id'] = properties_with_id_str['_id'].apply(pd.to_numeric, errors='coerce')
    properties_with_id_str = properties_with_id_str[~properties_with_id_str['_id'].isin(duplicate_properties_ok_ids)] 
    properties_with_id_str = properties_with_id_str.drop("", axis=1)

In [14]:
duplicate_properties_ok.head(20)

""


In [15]:
properties_with_id_str.head(20)

,_id,,add_date,address,city,company,description,img_src,link,plex-revenue,...,vendue,revenu,surface,longitude,latitude,construction_year,municipal_taxes,school_taxes,unites,geolocation


In [16]:
properties_to_rewrite= properties_with_id_str.to_dict('records') + duplicate_properties_ok.to_dict('records') 

import ast
for p in properties_to_rewrite:
    try:
        if str(p['geolocation']) !='nan' and 'coordinates' in p['geolocation']:
            geo = ast.literal_eval(p['geolocation'])
            p['longitude'] = geo['coordinates'][0]
            p['latitude'] =  geo['coordinates'][1]
        p.pop('geolocation')
    except Exception as e:
        print(str(p['geolocation']))
        print(f'{p['_id']} {e}')
        break
        

if len(properties_to_rewrite) > 0:
    pd.DataFrame(properties_to_rewrite).to_csv(f'properties_to_rewrite_{date.today().strftime("%d%m%Y")}.csv')
    rewrite_data(db='real_estate_db',collection='properties',data=properties_to_rewrite)

properties_to_rewrite = pd.read_csv(f'properties_to_rewrite_{date.today().strftime("%d%m%Y")}.csv').drop('Unnamed: 0', axis=1)#.to_dict('records')

select * from properties_to_rewrite
where _id not in (select _id from properties_db)